# Imports

In [1]:
#export
import os
import sys
sys.path.append(os.path.join(os.getcwd(),'exp'))

import numpy as np
from pprint import pprint

from skopt import Optimizer
from skopt.space import Real
#from bayes_opt import BayesianOptimization, UtilityFunction, SequentialDomainReductionTransformer

from config import cfg

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

# Code

In [3]:
#export
class BaseConfigBo:
    def __init__(self, n):
        self.n_points = n
        
    def get_values(self, o, names):
        o.update_next()
        set_points = o.ask(n_points=self.n_points)
        points = [{n:p for n,p in zip(sorted(names), points)} for points in set_points]
        return points
    
    def init_opt(self, bounds):
        return Optimizer(
                    dimensions=bounds,
                    random_state=1,
                    base_estimator='gp',
                    n_initial_points=2*self.n_points)
        
    def register(self, o, hp_points):
        NEGATIVE = -1
        for hp_point in hp_points:
            points = hp_point['points']
            x = [points[k] for k in sorted(points)]
            y = NEGATIVE * hp_point['target']
            o.tell(x, y)
            print(f'Registrating: {x}, {y}')
            
    def init_map(self):
        raise NotImplementedError
    
    def read_params(self, params):
        bounds = {}
        space_type = Real
        space_args = {'prior':'uniform', 'transform':None}
        for p in params:
            space_args['low'],space_args['high'] = p['bounds'][0], p['bounds'][1]
            bounds[p['name']] = space_type(**space_args)
        
        bounds = [bounds[k] for k in sorted(bounds)]
        return bounds
    
    def create_state(self, points, params, idx):
        names = [p['name'] for p in params]
        bounds = self.read_params(params)
        o = self.init_opt(bounds)

        if points:
            self.register(o, points)
        
        new_params = self.get_values(o, names)
        pprint(new_params, indent=4)
        new_params = new_params[idx]
        
        params_map = self.init_map()
        cfg = {}
        for name, (full_name, p_type, default_value) in params_map.items():
            value = new_params.get(name, default_value)
            if value is not np.NaN:
                cfg[full_name] = p_type(value)
        return new_params, cfg

# Tests

In [64]:
# points = [{'params':{'e':1,  'a':2},'target':0.1},
#           {'params':{'e':1,  'a':2},'target':0.2},
#           {'params':{'e':1,'a':2},'target':0.5},
#           {'params':{'e':1.2,'d':2},'target':0.0},
#           {'params':{'e':330,'k':2},'target':0.2}
#          ]

points = [
    {'params': {'cr': 0.9885257859302491}, 'target': 0.577763631939888},
    {'params': {'cr': 0.9885660456872435}, 'target': 0.5584937930107117},
    {'params': {'cr': 0.010119437482166772}, 'target': 0.5518008545041084},
    {'params': {'cr': 0.0101559066291012}, 'target': 0.5431795716285706},
    {'params': {'cr': 0.9899404424902588}, 'target': 0.5732786804437637},
    {'params': {'cr': 0.9893949599413431}, 'target': 0.5737223923206329},
    {'params': {'cr': 0.9898670833454175}, 'target': 0.5675030797719955},
    {'params': {'cr': 0.3092491872609939}, 'target': 0.5605851411819458},
    {'params': {'cr': 0.6836879323470609}, 'target': 0.567135363817215},
    {'params': {'cr': 0.5175112773164423}, 'target': 0.5583183616399765},
    {'params': {'cr': 0.010109605687386128}, 'target': 0.5333066955208778}
]

In [9]:
bounds = [Real(10,15, transform='normalize')]

In [11]:
o = Optimizer(   dimensions=bounds,
            random_state=1,
            base_estimator='gp',
            n_initial_points=3)

In [22]:
o.ask()

[14.231554583430086]

In [18]:
o.tell([11.982903636480131], 1)

          fun: 1
    func_vals: array([1])
       models: []
 random_state: RandomState(MT19937) at 0x7F9C8F01F048
        space: Space([Real(low=10, high=15, prior='uniform', transform='normalize')])
        specs: None
            x: [11.982903636480131]
      x_iters: [[11.982903636480131]]

In [65]:
from random import shuffle

In [66]:
p1 = {'name':'cr', 'bounds':(0.01,.99)}
shuffle(points)

# More Tests

In [ ]:
class Bo(BaseConfigBo):
    def __init__(self, params):
        self.params = [
            {'name':'exp_power', 'bounds':(1,15), 'type':int,},
            {'name':'crossover_chance', 'bounds':(0.01,0.99), },
        ]
        
        
    def init_map(self):
        return {
            'g':('generations', int, 2),
            'e':('exp_power', int, np.NaN),
            'f0':('dec_f0', int, np.NaN),
            'f1':('dec_f1', int, np.NaN),
            'f2':('dec_f2', int, np.NaN),
            'f3':('dec_f3', int, np.NaN),
            'mc':('mutate_chance', float, np.NaN),
            'cr':('crossover_chance', float, np.NaN),
            'co':('combine_chance', float, np.NaN)
        }

n_parallel_processes = len(cfg.GPUS.IDS)
bo = Bo(n_parallel_processes)
p1 = {'name':'e', 'bounds':(1,15)}
p2 = {'name':'cr', 'bounds':(0.01,0.99)}

def bo_all(**kwargs):
    inner_state, new_state=bo.create_state(points=kwargs['hp_points'], params=[p1, p2], idx=kwargs['idx'])
    return run(new_state=new_state, inner_state=inner_state, **kwargs)

def bo_exp(**kwargs):
    inner_state, new_state=bo.create_state(points=kwargs['hp_points'], params=[p1], idx=kwargs['idx'])
    return run(new_state=new_state, inner_state=inner_state, **kwargs)


In [4]:
class Bo(BaseConfigBo):
    def init_map(self):
        return {
            'g':('generations', int, 2),
            'e':('exp_power', int, np.NaN),
            'f0':('dec_f0', int, np.NaN),
            'f1':('dec_f1', int, np.NaN),
            'f2':('dec_f2', int, np.NaN),
            'f3':('dec_f3', int, np.NaN),
            'mc':('mutate_chance', float, np.NaN),
            'cr':('crossover_chance', float, np.NaN),
            'co':('combine_chance', float, np.NaN)
        }

n_parallel_processes = len(cfg.GPUS.IDS)
bo = Bo(n_parallel_processes)
p1 = {'name':'e', 'bounds':(1,15)}
p2 = {'name':'cr', 'bounds':(0.01,0.99)}

def bo_all(**kwargs):
    inner_state, new_state=bo.create_state(points=kwargs['hp_points'], params=[p1, p2], idx=kwargs['idx'])
    return run(new_state=new_state, inner_state=inner_state, **kwargs)

def bo_exp(**kwargs):
    inner_state, new_state=bo.create_state(points=kwargs['hp_points'], params=[p1], idx=kwargs['idx'])
    return run(new_state=new_state, inner_state=inner_state, **kwargs)


In [ ]:
Real(transform)

In [5]:
points = [{'points':{'cr': 0.49331196425848295, 'e': 13.444955669603809}, 'target':0.3},
         { 'points':{'cr': 0.6985920925954717, 'e': 1.8631242453991255}, 'target':.4},
           { 'points':{'cr': 0.6798862910768056, 'e': 5.684491632815158}, 'target':.4},
         ]
#points=None

In [37]:
new, ncfg = bo.create_state(points=points, params=[p1,p2], idx=2)

reging: [0.49331196425848295, 13.444955669603809], -0.3
reging: [0.6985920925954717, 1.8631242453991255], -0.4
reging: [0.6798862910768056, 5.684491632815158], -0.4
[   {'cr': 0.6985920925954717, 'e': 1.8631242453991255},
    {'cr': 0.37935403865037964, 'e': 3.0390075355652435},
    {'cr': 0.49331196425848295, 'e': 13.444955669603809}]


In [33]:
o.get_result()

          fun: 0.4
    func_vals: array([0.4])
       models: [GaussianProcessRegressor(alpha=1e-10, copy_X_train=True,
                         kernel=1**2 * Matern(length_scale=[1, 1], nu=2.5) + WhiteKernel(noise_level=1),
                         n_restarts_optimizer=2, noise='gaussian',
                         normalize_y=True, optimizer='fmin_l_bfgs_b',
                         random_state=1791095845)]
 random_state: RandomState(MT19937) at 0x7F3D0B706150
        space: Space([Real(low=0.01, high=0.99, prior='uniform', transform='normalize'),
       Real(low=1, high=15, prior='uniform', transform='normalize')])
        specs: None
            x: [0.6985920925954717, 1.8631242453991255]
      x_iters: [[0.6985920925954717, 1.8631242453991255]]

In [99]:
o.ask(1)

[[0.8237910319631421, 1.188286707127822]]

In [54]:
o.update_next()

In [ ]:
for i in range(10):
    x = optimizer.ask(n_points=4)  # x is a list of n_points points
    y = Parallel(n_jobs=4)(delayed(branin)(v) for v in x)  # evaluate points in parallel
    optimizer.tell(x, y)

# takes ~ 20 sec to get here
print(min(optimizer.yi))  # print the best objective found

# Exports

In [25]:
!python3 extra/n2s.py bo.ipynb

Converted bo.ipynb to exp/nb_bo.py
